In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import pandas as pd
from elasticsearch import Elasticsearch, helpers



In [2]:
es = Elasticsearch('http://localhost:9200')
df = pd.read_csv('./data/All_Novel.txt', index_col=0, parse_dates=['마지막 연재일', '연재 시작일'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9734 entries, 0 to 9733
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   제목       9734 non-null   object        
 1   장르       9734 non-null   object        
 2   작가       9734 non-null   object        
 3   연재수      9734 non-null   int64         
 4   연재 시작일   9734 non-null   datetime64[ns]
 5   마지막 연재일  9734 non-null   datetime64[ns]
 6   Site     9734 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 608.4+ KB


In [3]:
# orient='records' 를 사용해서 df의 행하나를 컬럼을 키값으로 가지고
#  행 하나를 value로 같는 리스트 형태의 딕셔너리를 만듬
docs = df.to_dict(orient='records') 
print(docs[0])
print(docs[0]['제목'])
print(len(docs))
print(docs[0].keys())

{'제목': ' 차라리 빌런으로 살겠다', '장르': '현대판타지', '작가': '서인하', '연재수': 127, '연재 시작일': Timestamp('2024-03-02 00:00:00'), '마지막 연재일': Timestamp('2024-06-25 00:00:00'), 'Site': 'Munpia'}
 차라리 빌런으로 살겠다
9734
dict_keys(['제목', '장르', '작가', '연재수', '연재 시작일', '마지막 연재일', 'Site'])


In [4]:
doc_list = []
i = 0
for dic in docs:
    i+=1

    doc = {
        '_index': 'web_novels',
        '_source': {
            'Title': dic['제목'],
            'Genre': dic['장르'],
            'Author': dic['작가'],
            'Count': dic['연재수'],
            'Site': dic['Site'],
            'period': {
                'start': dic['연재 시작일'],
                'last': dic['마지막 연재일']
            }
        }
    }
    doc_list.append(doc)

helpers.bulk(es, doc_list)    


(9734, [])

In [5]:
es = Elasticsearch('http://localhost:9200')
kdf = pd.read_csv('./data/K_all.txt', index_col=0, parse_dates=['마지막 연재일', '연재 시작일'])
kdf.info()
kdocs = kdf.to_dict(orient='records') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260 entries, 0 to 2259
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   제목       2260 non-null   object        
 1   장르       2260 non-null   object        
 2   작가       2260 non-null   object        
 3   조회수      2260 non-null   int64         
 4   댓글       2260 non-null   int64         
 5   연재수      2260 non-null   int64         
 6   연재 시작일   2260 non-null   datetime64[ns]
 7   마지막 연재일  2260 non-null   datetime64[ns]
 8   평점       2260 non-null   float64       
 9   Site     2260 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(4)
memory usage: 194.2+ KB


: 

In [6]:
i = 0
for kdoc in kdocs:
    i += 1
    es.index(index='kakao_novels', id=i, body=kdoc)

In [ ]:
es = Elasticsearch('http://localhost:9200')
mdf = pd.read_csv('./data/M_all.txt', index_col=0, parse_dates=['마지막 연재일', '연재 시작일'])
mdf.info()
mdocs = mdf.to_dict(orient='records') 

In [ ]:
i = 0
for mdoc in mdocs:
    i+=1
    es.index(index='munpia_novels', id=i, body=mdoc)

In [ ]:
es = Elasticsearch('http://localhost:9200')
ndf = pd.read_csv('./data/N_all.txt', index_col=0, parse_dates=['마지막 연재일', '연재 시작일'])
ndf.info()
ndocs = ndf.to_dict(orient='records') 


In [ ]:
ndoc_list = []
i = 0
for ndic in ndocs:
    i+=1

    ndoc = {
        '_index': 'naverseries_novels',
        '_source': {
            'Title': ndic['제목'],
            'Genre': ndic['장르'],
            'Author': ndic['작가'],
            'Count': ndic['연재수'],
            'Comments': ndic['댓글수'],
            'Favorite': ndic['추천수'],
            'Ratings': ndic['평점'],
            'Site': ndic['Site'],
            'period': {
                'start': ndic['연재 시작일'],
                'last': ndic['마지막 연재일']
            }
        }
    }
    ndoc_list.append(ndoc)

helpers.bulk(es, ndoc_list)    


In [ ]:
df = pd.read_csv('./data/All_Novel.txt', index_col=0)
df

In [ ]:
df.Site.unique()

In [ ]:
df[(df.장르 == '로맨스판타지') & (df.Site == 'NaverSeries')]